## Step 1 -  Reading Key Opinion Leaders Tweets
In this notebook we present the code that gets all the tweets of the Key opinion leaders between Jan 2014 - Feb 2020
This is step 1, In step 2 we will filter those tweets 

In [3]:
#! pip install inputimeout
# ! pip install tweeterid

In [1]:
import requests
import os
import json
import pandas as pd
import numpy as np
import csv , datetime, unicodedata, time, tweeterid #dateutil.parserm,
import gensim
import openpyxl
import json

from os import path
from datetime import datetime
from collections import Counter
from inputimeout import inputimeout, TimeoutOccurred

In [2]:
download_tweets = True #--> Change to True if you wish to download the tweets

In [3]:
from Brexit_Package import TwitterCrawler

my_token = 'NLP_is_fun' ### Of course one need to provide here a real valid token, unfortunelty we can't publish our token...
my_token = 'AAAAAAAAAAAAAAAAAAAAADRFOwEAAAAAaTp%2Bdd1OhobYMYb5ExPXm7IL6RA%3DDHknH402gOXoegUGxNtpC6giIjdackRLtdRx6tjrnnLeFN1ntT'
my_twitter_crawler = TwitterCrawler(bearer_token= my_token)


In [4]:
kol_list = ["theresa_may",
"NicolaSturgeon",
"MichelBarnier",
"donaldtusk",
"fhollande",
"PhilipHammondUK",
"DavidDavisMP",
"JunckerEU",
"guyverhofstadt",
"DidierSeeuws",
"EndaKennyTD",
"hilarybennmp",
"MinPres",
"MartinSelmayr",
"Keir_Starmer",
"realDonaldTrump",
"Wolf_Schauble",
"LiamFox",
"TimmermansEU",
"BorisJohnson",
"Nigel_Farage",
"NJ_Timothy",
"DacrePaul",
"ManfredWeber",
"davidmcallister",
"matteorenzi",
"PJWheatcroft",
"SadiqKhan",
"marianorajoy",
"HeywoodFndation",
"MartinSchulz",
"cbicarolyn",
"rupertmurdoch",
"WeyandSabine",
"OllyRobbins",
"MLP_officiel",
"MarkJCarney",
"FrancesOGrady",
"giannipittella",
"MalmstromEU",
"TheGinaMiller",
"CER_Grant",
"shriti_vadera",
"carlosghosn","David_Cameron",
"BSTimBarrow",
"AMCarwyn",
"WalkerWorcester",
"AlunCairns",
"DominicRaab",
"michaelgove",
"Dominic2306",
"theJeremyVine",
"George_Osborne",
"michaelhsltine_",
"WilliamJHague",
"ChukaUmunna",
"jeremycorbyn",
"Dyson",
"SteveO_Connell",
"jim_ratcliffe",
"MorrisseyHelena",
"GroovyTimbo", 
]

print("Number of KOL in the list = ", len(set(kol_list)))

Number of KOL in the list =  63


## Retrive tweets of Key Opinion Leaders
In the following code we are reading all the tweets of the list of the key opinion leaders in the given time-frame
The codes checks if a given name has an existing csv file, if it does, then the code will remove that name from the list so we won't retrieve data again.

In [23]:
#kol_list = ["Nigel_Farage"]

In [5]:

all_key_opinion_leader_names = kol_list
start_time = "2014-01-01T00:00:00Z"
end_time = "2020-02-01T00:00:00Z"

#start_time = "2020-02-02T00:00:00Z"
#end_time = "2021-12-31T00:00:00Z"

max_results = 500
limit_amount_of_returned_tweets = 10000000
dir_name = "KOL tweets"
##############################################################################################3
#### we don't want to get data on key opinion leaders that we already searched for their tweets!
# thus we first need to see which of the users that are inserted in the list are **not** in the files we already have:
from os import listdir
from os.path import isfile, join

import os.path
try:
    os.mkdir(dir_name)
    print("creating directory", dir_name, "to insert all the tables of all the key opinion leaders")
except:
    print("The dir", dir_name ,"already exist")

    
users_we_have_data = [f.split(".")[0] for f in listdir(dir_name) if isfile(join(dir_name, f))]

key_opinion_leader_names = list(set(all_key_opinion_leader_names) - set(users_we_have_data))
print("Users to search their tweets:",key_opinion_leader_names)
print("-----------------------------------------------")
print("Users we already have data on:",list(set(all_key_opinion_leader_names).intersection(users_we_have_data)))



The dir KOL tweets already exist
Users to search their tweets: ['PJWheatcroft', 'Wolf_Schauble', 'jim_ratcliffe', 'realDonaldTrump', 'Dominic2306', 'shriti_vadera', 'MarkJCarney']
-----------------------------------------------
Users we already have data on: ['MinPres', 'NicolaSturgeon', 'theresa_may', 'MartinSelmayr', 'SadiqKhan', 'WalkerWorcester', 'ChukaUmunna', 'EndaKennyTD', 'JunckerEU', 'Keir_Starmer', 'michaelgove', 'HeywoodFndation', 'NJ_Timothy', 'OllyRobbins', 'SteveO_Connell', 'GroovyTimbo', 'MartinSchulz', 'ManfredWeber', 'MorrisseyHelena', 'DidierSeeuws', 'CER_Grant', 'rupertmurdoch', 'TimmermansEU', 'BorisJohnson', 'TheGinaMiller', 'marianorajoy', 'DacrePaul', 'cbicarolyn', 'MalmstromEU', 'AlunCairns', 'Nigel_Farage', 'LiamFox', 'guyverhofstadt', 'BSTimBarrow', 'AMCarwyn', 'DominicRaab', 'George_Osborne', 'FrancesOGrady', 'matteorenzi', 'MichelBarnier', 'Dyson', 'davidmcallister', 'DavidDavisMP', 'donaldtusk', 'David_Cameron', 'jeremycorbyn', 'WeyandSabine', 'giannipittel

In [9]:
key_opinion_leader_names = kol_list
dir_name = "KOL tweets 2021"

In [10]:
# ##########################################
query = "lang:en -is:retweet -is:reply -is:quote "

if download_tweets:
    my_twitter_crawler.return_tweets_of_key_opinion_leaders(query=query, user_names= key_opinion_leader_names,
                                            start_time = start_time,
                                            end_time = end_time,
                                            max_results = max_results, evaluate_last_token = False,
                                            limit_amount_of_returned_tweets = limit_amount_of_returned_tweets,
                                           verbose_10 = False, dir_name =dir_name)

Bringing tweets of theresa_may
creating directory KOL tweets 2021 to insert all the tables of all the key opinion leaders
Bringing all the tweets of the user: theresa_may from: 2014-01-01T00:00:00Z to 2020-02-01T00:00:00Z

creating directory KOL tweets 2021\log_key_opinion_leaders to insert all the logs of the key opinion leaders
creating directory KOL tweets 2021\log_key_opinion_leaders\theresa_may in the dir KOL tweets 2021\log_key_opinion_leaders to insert all the logs of the key opinion leader theresa_may
no more tweets from this user
Total amount of collected tweets =  1760
1760
---------------------------------------------------------------
Bringing tweets of NicolaSturgeon
The dir KOL tweets 2021 already exist
Bringing all the tweets of the user: NicolaSturgeon from: 2014-01-01T00:00:00Z to 2020-02-01T00:00:00Z

The dir KOL tweets 2021\log_key_opinion_leaders already exist
creating directory KOL tweets 2021\log_key_opinion_leaders\NicolaSturgeon in the dir KOL tweets 2021\log_ke

no more tweets from this user
Total amount of collected tweets =  4281
4281
---------------------------------------------------------------
Bringing tweets of realDonaldTrump
The dir KOL tweets 2021 already exist
Bringing all the tweets of the user: realDonaldTrump from: 2014-01-01T00:00:00Z to 2020-02-01T00:00:00Z

The dir KOL tweets 2021\log_key_opinion_leaders already exist
creating directory KOL tweets 2021\log_key_opinion_leaders\realDonaldTrump in the dir KOL tweets 2021\log_key_opinion_leaders to insert all the logs of the key opinion leader realDonaldTrump
no data / include in the json
no more tweets from this user
Total amount of collected tweets =  0
0
The user: realDonaldTrump had 0 tweets!!
---------------------------------------------------------------
Bringing tweets of Wolf_Schauble
The dir KOL tweets 2021 already exist
Bringing all the tweets of the user: Wolf_Schauble from: 2014-01-01T00:00:00Z to 2020-02-01T00:00:00Z

The dir KOL tweets 2021\log_key_opinion_leaders al

limit_output extension: Maximum message size of 10000 exceeded with 10112 characters